In [ ]:
# pip install sanction
# pip install lyrics genius

In [ ]:
import lyricsgenius as genius
import json
import pandas as pd
import os

In [73]:
FULL_ARTISTS = [
    "Bloody Hawk", "LEX (GRC)", "Logos Timis", "Eisvoleas", "Ethismos", "Negros tou Moria", "RICTA", "Immune (GRC)", 
    "12os Pithikos", "Vlospa", "Mikros Kleftis", "Tsaki", "Wang (GRC)",
    "Dani Gambino", "Katohos", "Gxhan (GRC)", "Novel 729", "Kareem Kalokoh", "Anser (GRC)", "Hawk (GRC)", "TOQUEL", 
    "Sidarta", "Jaul", "HGEMONA$", "FANN", "Voreia Asteria", "Mpelafon", "Buzz (GRC)", "Trouf", "Mad Clip", 
    "Light", "Sadam (GRC)", "Saske (GRC)", "GRIZZLE (GRC)", "Avi (GRC)", "Nume (GRC)",
    "Logos Apeili", "Sifu Versus", "Detro (GRC)", "Hatemost", "Lobo (GRC)", "ΔΠΘ | Deltapithita", "Sponty", "FLY LO (GRC)", 
    "Βέβηλος (Vevilos)", "Bong Da City", "Mente Fuerte", "Trannos", "Kidd (GRC)", "HermesHermes", "Ivan Greko", "Arab (GRC)", 
    "DAIMA", "Roi 6/12", "FY", "Lava (GRC)", "Ropex Laterio", "Strat", "RACK", "LILA (GRC)", "Sicario (GRC)", "Snik", 
    "Moose (GRC)", "Thug Slime", "YungKapa", "Half Quickie", "OGE (GRC)", "XRS (GRC)", "Skez", "Yanek (GRC)", 
    "Ghetto Queen (GRC)", "Tzamal", "Sin Boy", "APOF", "Diff (GRC)", "Sigma (GRC)", "ZiZZi", "Bossikan", "Taff",
    "Taf Lathos", "Jitano", "KOAS", "MG (GRC)", "Dirty Harry (CYP)", "Mc Daddy", "GMARK", "ATC Coco (GRC)", "Yolte (GRC)",
    "Onad (GRC)", "Block '93", "Scar (YM)", "Alecc (GRC)", "Long3", "Jolly Roger", "ATC Toro", "ATC Nico", "Ggreco (GRC)",
    "Iratus", "Zinon", "Kanon", "RNS (GRC)", "SUPREME (GRC)", "Mani (GRC)",
    "Rio (GRC)", "Μηδενιστής (Midenistis)", "Zontani Nekri", "Ypo", "Kataxthonios",
    "Taki Tsan", "Xarmanis", "Billy Sio", "Fi Vita Sigma", "Inka (GRC)", "Sakir",
    "X.T.P. (GRC)", "Joker/Two-Face", "Styl Mo", "Nesok", "ΙΖΩ (IZW)", "Arrwsto Pneuma", "Jako (GRC)",
    "Split (GRC)", "Ai Tsavouras (GRC)", "LADELE", "Expe (GRC)", "Aria (GR)", "AEON (GRC)", "Above The Hood",
    "Bitapeis", "Ημισκούμπρια (Imiskoubria)", "Anapoda Kapela", "BTK (GRC)"
]

In [ ]:
len(FULL_ARTISTS)

132

In [ ]:
genius = genius.Genius(
    API_TOKEN,
    remove_section_headers=False,
    skip_non_songs=True, # Skip pages that aren't actual songs
    verbose=False,
    timeout=15,  
    retries=3
)

In [ ]:
# --- 1. CONFIGURATION FOR THIS RUN ---
# Change these numbers for each manual batch!
# 90-98 check
# 0 - 99
START = 40
END = 50
FILENAME = 'genius_songs_data.csv'

# Slice the list to get only the artists for this specific run
current_batch = FULL_ARTISTS[START:END]
print(f"Processing batch from {START} to {END}...")

data = []

# --- 2. THE LOOP ---
for artist_name in current_batch:
    try:
        print(f"Searching for {artist_name}...")
        artist = genius.search_artist(artist_name, sort='popularity', get_full_info=True)
        
        for song in artist.songs:
            featured_artist_name = None
            if song.featured_artists:
                featured_artist_name = song.featured_artists[0]['name']

            # Use .to_dict() to get raw data
            song_details = song.to_dict()
            release_date = song_details.get('release_date')
            
            # Safe access for pageviews (avoids crash if 'stats' is missing)
            stats = song_details.get('stats')
            pageviews = stats.get('pageviews') if stats else None

            row = {
                'artist': song.artist,
                'title': song.title,
                'album': song.album['name'] if song.album else None,
                'annotation_count': song.annotation_count,
                'featured_artists': featured_artist_name,
                'lyrics': song.lyrics,
                'date': release_date,
                'pageviews': pageviews
            }
            data.append(row)
            
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")

# --- 3. SAVE TO CSV (APPEND MODE) ---
if data:
    df = pd.DataFrame(data)
    
    # Check if file exists so we know if we need to write the header
    file_exists = os.path.isfile(FILENAME)
    
    # mode='a' means append (add to end) rather than 'w' (overwrite)
    # header=not file_exists means: write header ONLY if file is new
    df.to_csv(FILENAME, mode='a', index=False, header=not file_exists)
    
    print(f"Successfully appended {len(df)} songs to {FILENAME}")
else:
    print("No data found in this batch.")

Processing batch from 40 to 50...
No data found in this batch.


In [66]:
df = pd.read_csv("genius_songs_data.csv")

In [67]:
df

,artist,title,album,annotation_count,featured_artists,lyrics,date,pageviews
0,Alecc (GRC),Marijuana,NaN,0,FY,[Chorus: Ggreco]\nΠίνω marijuana-na-na-na-na-n...,2023-11-02,NaN
1,Alecc (GRC),Malaga,NaN,0,NaN,[INTRO]\nΜάλαγα\nΜάλαγα\nΜα μα Μάλαγα\nΦεύγω σ...,2024-06-09,NaN
2,Alecc (GRC),Τι να Κάνω (Ti Na Kanw),NaN,1,NaN,[Intro]\nΈχω πεί ότι θα αλλάξω αλλά ποτέ δε θα...,2021-10-06,NaN
3,Alecc (GRC),Drill Life,NaN,0,NaN,"[Part 1]\n\n[Intro]\nAleccBaby, AleccBaby\nTra...",2022-04-08,NaN
4,Alecc (GRC),PLEIN,NaN,1,NaN,"[Chorus]\nΌλο με κοιτάει, κοζάρει τα Philipp P...",2025-03-06,NaN
...,...,...,...,...,...,...,...,...
6055,BTK (GRC),Δυο Ματιές (Dio Maties),Ευχές (Euxes),0,NaN,"[Chorus]\nΨάχνω τον τρόπο να φύγω, ψάχνεις τον...",2025-03-02,NaN
6056,BTK (GRC),Γειτονιές (Geitonies),NaN,0,NaN,[Intro]\nΈνα\nΒ-Τ-Κ\nΛέει\n\n[Verse 1]\nΑπό το...,2025-06-13,NaN
6057,BTK (GRC),Livera,Ευχές (Euxes),0,NaN,[Intro]\nΤα βήματα που έκανα ήταν βαριά\nΚάθε ...,2025-03-03,NaN
6058,BTK (GRC),Εμπόλεμη Ζώνη (Empolemi Zoni),Ευχές (Euxes),0,NaN,[Verse 1]\nΦτύνω στο μικρόφωνο να το ακούσουν ...,2025-03-02,NaN


In [69]:
df['artist'].nunique()

132